In [1]:
import numpy as np
import pandas as pd
import time
import os
import sys

## Utils

In [2]:
def read_file(filename: str):
    with open(filename) as file:
        books_in_library_with_id = []
        nb_books_in_library_with_id = []
        signup_days_for_library_with_id = []
        shipping_speed_for_library_with_id = []
        
        for idx, line in enumerate(file):
            if(idx == 0):
                # Start of the file
                line_1 = line.replace('\n', '').split(' ')
#                 print(line_1)
                nb_books = int(line_1[0])
                nb_libraries = int(line_1[1])
                nb_days = int(line_1[2])

#                 print(nb_books)
#                 print(nb_libraries)
#                 print(nb_days)
            elif(idx == 1):
                book_scores_with_id = line.replace('\n', '').split(' ')
                book_scores_with_id = list(map(int,book_scores_with_id))

            else:

                if(idx % 2 == 0):
                    
                    #### Per library section
                    line_1 = line.replace('\n', '').split(' ')
                    if(line_1[0] == ''):
                        break

                    nb_books_in_library_with_id.append(int(line_1[0]))
                    signup_days_for_library_with_id.append(int(line_1[1]))
                    shipping_speed_for_library_with_id.append(int(line_1[2]))
                    
#                     print(nb_books_in_library_with_id)
#                     print(signup_days_for_library_with_id)
#                     print(shipping_speed_for_library_with_id)
                if(idx % 2 == 1):

                    book_ids = line.replace('\n', '').split(' ')
                    if(book_ids[0] == ''):
                        break

                    book_ids = list(map(int,book_ids))
                    books_in_library_with_id.append(book_ids)
#                     print("book ids : {}".format(books_in_library_with_id))



    return nb_books, \
            nb_libraries, \
            nb_days, \
            nb_books_in_library_with_id, \
            signup_days_for_library_with_id, \
            shipping_speed_for_library_with_id, \
            book_scores_with_id ,\
            books_in_library_with_id

In [3]:
def create_submission(nb_pizzas: int, pizza_nb_list: list, directory: str, output_file: str) -> None:
    TODO

## Get Score

In [10]:
def get_score_from_library(library_id : int, days_left : int, books_already_scanned : list, signup_days_for_library_with_id : list, book_scores_with_id: list):

    # Update days left
    days_left = days_left - int(signup_days_for_library_with_id[library_id])

    # Compute books scanned and possible books in library to be used
    books = books_in_library_with_id[library_id]
    books_already_scanned_set = set(books_already_scanned)
    possible_books = set(books).difference(books_already_scanned_set)
    books_scanned = books_already_scanned_set.union(possible_books)

    # Compute score
    score = 0
    for book in possible_books:
        score += book_scores_with_id[book]

    return score, days_left, list(books_scanned)

## Recursive algorithm

In [11]:
# Recursive algorithm; we update the max recursion depth.
# TODO; Change to iterative algo
sys.setrecursionlimit(15000)

In [32]:
def select_promising_libraries(Libraries):
    return Libraries[:3]

In [33]:
def solve(Libraries, nb_day, books_already_scanned, signup_days_for_library_with_id, book_scores_with_id, depth):
    
    if(depth>10): 
        """Pruning"""
        return 0 
        """TODO; update this"""
    elif(len(Libraries) == 0):
        return 0
    elif(nb_day == 0):
        return 0
    elif(nb_day<0):
        return None
    elif(len(Libraries) < 0):
        raise Exception("PROB")
    else:
        scores = []
        node_score, days, books = get_score_from_library(0 ,nb_day, books_already_scanned, signup_days_for_library_with_id, book_scores_with_id)

        for i in range(len(select_promising_libraries(Libraries))):

            sub_solution = solve(Libraries[:i] + Libraries[i+1:], nb_day - int(signup_days_for_library_with_id[i]), set(books).union(books_already_scanned), 
                                 signup_days_for_library_with_id[:i] + signup_days_for_library_with_id[i+1:],
                                 book_scores_with_id[:i] + book_scores_with_id[i+1:], depth+1)
            if sub_solution is not None:
                scores.append(sub_solution)
                
        if scores == []:
            return None
        else:
            return (max(scores)+node_score)    

## Main

In [34]:
file_names = ["data/a_example.txt", "data/b_read_on.txt", 
              "data/c_incunabula.txt", "data/d_tough_choices.txt", 
              "data/e_so_many_books.txt", "data/f_libraries_of_the_world.txt"]

In [35]:
file_name = file_names[1]

nb_books , \
	nb_libraries, \
	nb_days, \
	nb_books_in_library_with_id, \
	signup_days_for_library_with_id, \
	shipping_speed_for_library_with_id, \
	book_scores_with_id ,\
	books_in_library_with_id  = read_file(file_name)

libraries = list(range(nb_libraries))

start_time = time.time()
print(solve(libraries, nb_days, set(), signup_days_for_library_with_id, book_scores_with_id, 0))
print(time.time() - start_time)

100000
1395.641329050064
